<img src="https://raw.githubusercontent.com/AdrianoPereira/CAP421/main/lectures/homework11/images/cover-homework11.png" style="width: 100%;">

### OVERVIEW
<hr />

This notebook contains exercise 11 (optional exercise 02). This exercise was proposed by Professor Valdivino Santiago Júnior in the course CAP421 - Deep Learning offered in the Postgraduate Program in Applied Computing at the National Institute for Space Research.

**Exercise optional 02:**
> Explain why the "new" proposed DenseNet-83 has more trainable parameters and demands more memory (parameters) than the DenseNet-121. Create a simple network with some dense blocks (enventually transition layers), calculate the number os trainable parameters and related memory to solve this exercise.


**Author:** Adriano P. Almeida <<adriano.almeida@inpe.br>>
<br>
**Created on:** 12 November, 2021
<br /><br />

<a href="#">
    <img style="float: left; margin-right: 10px;" src="https://colab.research.google.com/assets/colab-badge.svg" />
</a>

<a href="https://github.com/AdrianoPereira/CAP421/tree/main/lectures/homework11">
    <img style="float: left; margin-right: 10px;" src="https://img.shields.io/badge/GitHub-Open%20Repository-lightgrey?logo=github" />
</a>

## 1. DenseNet
<hr />

A DenseNet é um arquitetura de rede neural convolucional em que as camadas ocultas possuem conexões com as camadas subsequentes, com um certo grau de persistência [(HUANG, 2017)](#huang_densely_2017). Essa característica é semelhante a que esta presente na arquitetura ResNet [(HE, 2016)](#he_deep_2016), porém, existem diferenças em como essas conexões são feitas. A ResNet realiza uma operação de soma com as informações disponíveis nas camadas, que pode ser descrita como $a^{l} = g(a^{[l+1]}+a^{[1]})$, em que $g$ é uma função da taxa de alcance da rede neural. Já a DenseNet realiza uma concatenação das conxeções, descrita como $a^{l} = g(a^{[0]}, a^{[1]}, ..., a^{[l-1]})$. Essas conexões permitem que a rede não tenha o problema de desaparecimento de gradiente, uma vez que as informações são transferidas para mais de uma camada, como pode ser observado na Figura [1](#densenet).

<p>
<img width="60%" src="https://raw.githubusercontent.com/AdrianoPereira/CAP421/main/lectures/homework11/images/densenet.png" />
<br />
<span style="display: block; text-align: center;">
<strong>Figura 1</strong>: Esquema de um bloco da arquitetura DenseNet. <strong>Fonte</strong>:<a href=""#huand_densely_2017">Huan (2017)</a>
</span>
</p>
    
O número de conexões diretas da DenseNet é dado po $\frac{L(L+1)}{2}$, em que $L$ é o número de camadas. Existem versões da DenseNet com diferentes números de camadas, tais como 121, 160, 201, etc. Esse número de camadas é dado por blocos que contém filtros convolucionais, camadas de transição e de classificação. Por exemplo, DenseNet-121 possui 2 blocos com 5 camadas convolucionis e de sub-amostragem (*pooling*), 42 camadas de  transição, 16 camadas de classificação.

In [5]:
import tensorflow as tf


base_densenet121 = tf.keras.applications.DenseNet121(include_top=False, weights='imagenet')
base_densenet121.summary()

Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
________________________________________________________________________________________

In [14]:
def get_memory_usage_in_MB(model, *, batch_size: int):
    default_dtype = tf.keras.backend.floatx()
    shapes_mem_count = 0
    internal_model_mem_count = 0
    for layer in model.layers:
        if isinstance(layer, tf.keras.Model):
            internal_model_mem_count += keras_model_memory_usage_in_bytes(
                layer, batch_size=batch_size
            )
        single_layer_mem = tf.as_dtype(layer.dtype or default_dtype).size
        out_shape = layer.output_shape
        if isinstance(out_shape, list):
            out_shape = out_shape[0]
        for s in out_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = sum(
        [tf.keras.backend.count_params(p) for p in model.trainable_weights]
    )
    non_trainable_count = sum(
        [tf.keras.backend.count_params(p) for p in model.non_trainable_weights]
    )

    total_memory = (
        batch_size * shapes_mem_count
        + internal_model_mem_count
        + trainable_count
        + non_trainable_count
    )
    return total_memory

memory_usage = get_memory_usage_in_bytes(model=base_densenet121, batch_size=32)/1024/1024
print(f"{memory_usage:.3f} MB")

22.372 MB
